In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from Resoruces.utils import CryptoDataset
from torch.utils.tensorboard import SummaryWriter
from Models.AttendingLSTM import AttendingLSTM
from torch.utils.data import Subset
from tqdm import tqdm
import os

seq_len = 20

train_dataset = CryptoDataset(csv_file="./Data/train.csv",
                                seq_len=seq_len,
                                is_train=True)

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("cuda")
else:
    DEVICE = torch.device("cpu")
    print("cpu")

AttnLSTM = AttendingLSTM(9,9,9)
AttnLSTM = AttnLSTM.to(DEVICE)
loss_fn = nn.BCELoss()
optimizer = optim.AdamW(AttnLSTM.parameters(), lr=2e-3)  

epoch_count = 1000
train_len = 10000
valid_len = 10000
OUTPUT_FILEID = "AttendingLSTM_Test"

subset_indices = list(range(100000 - seq_len))  # Indices to keep

train_dataset = Subset(train_dataset, subset_indices)

train,valid = torch.utils.data.random_split(train_dataset, [0.8, 0.2])

training_loader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
validation_loader = torch.utils.data.DataLoader(valid, batch_size=16, shuffle=False)


tb_writer = SummaryWriter(
    log_dir=f"./log/{OUTPUT_FILEID}",
    max_queue=5
)

print(sum(p.numel() for p in AttnLSTM.parameters()))

cpu
2413


In [2]:
from torchviz import make_dot
import os

os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'
data = next(iter(training_loader))
yhat = AttnLSTM(data['values'].to(DEVICE),data['values'].to(DEVICE)) # 
make_dot(yhat, params=dict(list(AttnLSTM.named_parameters()))).render("attn_lstm_torchviz", format="png")

C:\Users\gabes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\_tensor.py:955: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


'attn_lstm_torchviz.png'

In [ ]:


for epoch in range(epoch_count):
    running_loss = 0.
    running_valid_loss = 0.
    last_loss = 0.
    last_valid_loss = 0.
    best_loss = 1000.
    best_valid = 0.
    AttnLSTM.train()
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in tqdm(enumerate(training_loader),total= len(training_loader)):
        # Every data instance is an input + label pair
        time = data['time'].to(DEVICE)
        values = data['values'].to(DEVICE)
        target = data['target'].to(DEVICE)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = AttnLSTM.forward(time,values)
      
        # Compute the loss and its gradients
        loss = loss_fn(outputs, target)
        
        loss.backward()
        
        # Adjust learning weights
        optimizer.step()
        # Gather data and report
        running_loss += loss.item()

        if i % 1000 == 999:
            
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    AttnLSTM.eval()
    for i, data in tqdm(enumerate(validation_loader),total= len(validation_loader)):
        time = data['time'].to(DEVICE)
        values = data['values'].to(DEVICE)
        target = data['target'].to(DEVICE)

        # Zero your gradients for every batch!
        optimizer.zero_grad()
        
        # Make predictions for this batch
        outputs = AttnLSTM.forward(time,values)
        
        # Compute the loss and its gradients
        loss = loss_fn(outputs, target)
        running_valid_loss += loss.item()
        if i % 1000 == 999:
            
            if best_valid > last_loss: 
                best_valid = last_loss
            last_loss = running_valid_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/valid', last_loss, tb_x)
            print(last_loss)
            running_valid_loss = 0.

    os.makedirs(f"./data/model", exist_ok=True)


    model_to_save = AttnLSTM
    weights = model_to_save.state_dict()#prefix_mlp(model_to_save.prefix_wte.weight)

    checkpoint = weights

    # save current model
    torch.save(
        checkpoint,
        f"./data/model/{OUTPUT_FILEID}.checkpoint"
    )
    

    # save best model
    if best_valid < best_loss:
        best_loss = best_valid
        torch.save(
            checkpoint,
            f"./data/model/{OUTPUT_FILEID}.best.checkpoint"
        )


    


  0%|          | 2/4999 [00:00<07:04, 11.77it/s]

7.123212814331055
9.302949905395508
9.068167686462402
5.4956278800964355


  0%|          | 6/4999 [00:00<05:16, 15.80it/s]

8.564460754394531
8.328962326049805
8.110608100891113
4.879322528839111


  0%|          | 14/4999 [00:00<03:44, 22.16it/s]

...

 29%|██▉       | 1454/4999 [00:46<01:58, 29.85it/s]

0.5003193616867065
0.7424278855323792
0.8516782522201538
0.8592503666877747
0.6610052585601807
0.7140682935714722


 29%|██▉       | 1458/4999 [00:47<01:56, 30.40it/s]

0.693015992641449
0.7201426029205322
0.7554818987846375
0.7492926716804504
0.6722257733345032
0.8076995611190796
0.7300553917884827
0.7160214781761169


 29%|██▉       | 1462/4999 [00:47<01:51, 31.67it/s]